### TODO

## Thread functions

In [1]:
from lib.multitasking_lib import *
#from lib.kite_helper_lib import *

In [2]:
freedom = threadManager("freedom", [ "kite_simulator1", "trade_handler1"], 
                        [kite_simulator, trade_handler])

In [6]:
freedom.job.terminate()

In [8]:
print(freedom.job.is_alive())

False


In [4]:
freedom.add("order_handler1", order_handler)

In [ ]:

stock = 'ONGC'
logger.setLevel(logging.DEBUG)
trade_analysis(stock)

In [ ]:
import dash_html_components as html

print(html.Br())

In [ ]:
# Long Strategies
def long_indicators(s):
    temp_df = 0
    #temp_df = temp_df | (REF(s.fastk,1) <=20)&(s.fastk>20)
    #temp_df = temp_df | CROSSOVER(s.fastk, s.fastd)
    #temp_df = temp_df | (REF(s.macd,1) <=0)&(s.macd>0)
    #temp_df = temp_df | CROSSOVER(s.macd, s.macdsignal)
    temp_df = temp_df | (REF(s.rsi,1) <=40)
    #temp_df = temp_df | (s.CLOSE.shift(1) == s.min.shift(1)) & (s.CLOSE > s.OPEN)
    #temp_df = temp_df & CROSSOVER(s.BBB, s.CLOSE)

    return pd.DataFrame(  temp_df, columns=["buy"] )

# Short Strategies
def short_indicators(s):
    temp_df = 0
    #temp_df = temp_df | (REF(s.fastk,1) >=80)&(s.fastk<80)
    #temp_df = temp_df | CROSSOVER(s.fastd, s.fastk)
    #temp_df = temp_df | (REF(s.macd,1) >=0)&(s.macd<0)
    #temp_df = temp_df | CROSSOVER(s.macdsignal, s.macd)
    temp_df = temp_df | (REF(s.rsi,1) >=60)
    #temp_df = temp_df & CROSSOVER(s.CLOSE, s.BBT)
    return pd.DataFrame( temp_df, columns=["sell"])



def long_breakout(s):
    temp_df = s.haCLOSE >= s.BBT.shift(1)
    temp_df = temp_df | (s.haCLOSE >= s.haOPEN.shift(2))
    temp_df = temp_df & ( CROSSOVER(s.OPEN, s.BBT) | CROSSOVER(s.OPEN, s.BBB) | CROSSOVER(s.OPEN, s.BBM))
    return pd.DataFrame( temp_df , columns=["buy"])

def short_breakout(s):
    temp_df = s.haOPEN <= s.BBB.shift(1)
    temp_df = temp_df | (s.haCLOSE <= s.haOPEN.shift(2))
    temp_df = temp_df & ( CROSSOVER(s.BBT, s.OPEN) | CROSSOVER(s.BBB, s.OPEN) | CROSSOVER(s.BBM, s.OPEN))
    return pd.DataFrame( temp_df , columns=["sell"])

def long_ha(s):
    temp_df = (REF(s.haCLOSE,3) < REF(s.haOPEN,3)) & (REF(s.haCLOSE,2) < REF(s.haOPEN,2)) & (REF(s.haCLOSE,1) > REF(s.haOPEN,1)) & (s.haCLOSE > s.haOPEN)  
    #temp_df = temp_df & (s.rsi < 40 )
    return pd.DataFrame( temp_df , columns=["buy"])

def short_ha(s):
    temp_df = (REF(s.haCLOSE,3) > REF(s.haOPEN,3)) & (REF(s.haCLOSE,2) > REF(s.haOPEN,2)) & (REF(s.haCLOSE,1) < REF(s.haOPEN,1)) & (s.haCLOSE < s.haOPEN)
    #temp_df = temp_df & (s.rsi > 60 )
    return pd.DataFrame( temp_df , columns=["sell"])
    


## Definition of handler functions

In [ ]:
#freedom_init = threadManager("freedom_init", ["freedom_init"], [freedom_init])   

#freedom = threadManager("freedom", ["user_requests_handler", "kite_simulator", "backtest_handler", "trade_handler","order_handler"], 
#                        [user_requests_handler, kite_simulator, backtest_handler, trade_handler, order_handler])

logger.setLevel(21)
freedom = threadManager("freedom", [ "kite_simulator", "trade_handler", "order_handler"], 
                        [kite_simulator, trade_handler, order_handler])


In [ ]:
# Shut down
conn.publish('kite_simulator','stop')
conn.publish('trade_handler','stop')
freedom.job.terminate()
print(freedom.job.is_alive())
conn.pubsub_channels()

## Redis Console

In [ ]:
from redis import Redis
import json

conn = Redis(host='redis', port=6379, db=0, charset="utf-8", decode_responses=True)

In [ ]:

def trade_analysis(stock):
    trade_log = pd.read_json(conn.get(stock+'Trade'))

    state = 'None'
    trade_log['profit'] = 0
    profit = 0
    total_win = 0
    total_loss = 0
    max_profit = 0 
    max_loss = 0
    winning_streak = 0
    loosing_stream = 0
    prev_profit = 0
    winning_streak = 1
    loosing_streak = 1
    max_winning_streak = 0
    max_loosing_streak = 0

    for index, row in trade_log.iterrows():
        if not math.isnan(row.buy):
            profit -= row['buy']
            #print(row['buy'])  
        if not math.isnan(row.sell):
            profit += row['sell']
            #print(row['sell'])

        if state=='None':
            trade_log.loc[index,'profit'] = 0
            state='Trade'
        else:
            trade_log.loc[index,'profit'] = profit
            if profit >=0:
                total_win += 1
                if profit > max_profit:
                    max_profit = profit

                if prev_profit > 0:
                    winning_streak += 1
                elif loosing_streak > max_loosing_streak:
                    max_loosing_streak = loosing_streak
                loosing_streak = 1    

            else:
                total_loss += 1
                if profit < max_loss:
                    max_loss = profit

                if prev_profit < 0:
                    loosing_streak += 1
                elif winning_streak > max_winning_streak:
                    max_winning_streak = winning_streak
                winning_streak = 1

            prev_profit = profit
            profit = 0
            state = 'None'

    total_profit = trade_log.profit.sum()
    trade_log['CumProfit'] = trade_log.profit.cumsum()
    logtrade('Profit: {}, max loss: {}, max win: {}'.format(total_profit, max_loss, max_profit))
    logtrade('# of Win: {}, # of Loss: {}, max_winning_streak: {}, max_loosing_streak: {}'.format(total_win, total_loss, max_winning_streak, max_loosing_streak))

    logtrade('\n{}'.format(trade_log.fillna('')))

In [ ]:
import pandas as pd


stock = 'ONGC'
logger.setLevel(logging.DEBUG)
trade_analysis(stock)

In [ ]:
conn.xadd('msgBufferQueue',{'msg': 'next'})

In [ ]:

# Unit Test#1
freq='day'
exchange='NSE'
fromDate = '2019-01-01'
toDate = '2019-05-03'
stock='INFY'

state = 'INIT'
hash_key = stock+'_state'

try:
    all_keys = list(conn.hgetall(hash_key).keys())
    conn.hdel(hash_key,*all_keys)
except:
    pass

#conn.hmset(hash_key, {'state':state,'stock':stock, 'qty':0,'price':0,'algo':'','freq':'day','so':0,'target':0})

msg = json.dumps({'stock': stock, 'fromDate':fromDate,'toDate':toDate, 'exchange':exchange, 'freq':freq})
conn.publish('trade_handler','start')
msgid2 = conn.publish('kite_simulator',msg)


In [ ]:
# Unit Test#2
stock = 'NDTV'
msg = json.dumps({'stock': stock, 'fromDate':fromDate,'toDate':toDate, 'exchange':exchange, 'freq':freq})
conn.publish('trade_handler','start')
msgid2 = conn.publish('kite_simulator',msg)
conn.xadd('msgBufferQueue',{'msg': 'next'})

In [ ]:
# Shut down
conn.publish('kite_simulator','stop')
conn.publish('trade_handler','stop')
freedom.job.terminate()
print(freedom.job.is_alive())
conn.pubsub_channels()

In [ ]:
conn.pubsub_channels()

In [ ]:
# Debug
stock='TCS'
#notification_despatcher(None, 'msg')
print(stock)
pd.read_json(conn.hget(stock+'_state','ohlc'))

In [ ]:
pd.set_option('display.max_rows', 50)
pd.read_json(conn.hget(stock+'_state','ohlc')).tail(50)

## Scratchpad

In [ ]:
import pandas as pd
import plotly
import plotly.graph_objs as go
from datetime import datetime as dt
from datetime import timedelta
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from lib.multitasking_lib import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [ ]:
# GetData

logger.setLevel(logging.DEBUG)
stock = 'WIPRO'
fromDate = '2019-2-2'
toDate  = '2019-2-28'
freq = 'minute'
exchange = 'NSE'
ohlc_data = {}
ohlc_data[stock] = getData(stock, fromDate, toDate, exchange, freq, False, stock)

In [ ]:
import numpy as np
no = 1
for i in np.linspace(0,no-1,no):
    i = int(i)
    row = ohlc_data[stock].iloc[i]
    print(type(row))
    index = ohlc_data[stock].index[i]
    update_plot_cache(stock, row)
    print(index)
    print(ohlc_data[stock].iloc[i])
    # Construct Json message like Kite
    mydate = "{}-{}-{}".format(index.year,index.month,index.day)        
    msg = {exchange+":"+stock:{"ohlc":{'date':mydate,'open':row['open'],'high':row['high'],'low':row['low'],'close':row['close'],'volume':row['volume']}}}
    pdebug(msg)
    msg = json.dumps(msg)

In [ ]:
for i,r in ohlc_data[stock].iterrows():
    print(type(r))
    print(index)
    print(r)
    break

In [ ]:
#tmp_df = pd.DataFrame({'buy': ohlc_data.iloc[1:5]['close'], 'sell':ohlc_data.iloc[10:15]['close']})

tmp_df_s = pd.DataFrame()
tmp_df_b = pd.DataFrame()
tmp_df_b['buy'] = ohlc_data.iloc[-1:]['close']
tmp_df_s['sell'] = ohlc_data.iloc[20:25]['close']

tmp_df_b.append(tmp_df_s)